# 例外

正常なデータを使用して正常なルートを通ることを考えてプログラミングしていませんか？  
もちろんそれが一番理想的ではありますが、実際には状況によっては予期しない事象で正しく処理が行えないことがあります。  
予期しない状況をJavaでは例外(Exception)と呼びます。  
想定外の事象をなるべく減らすためには、例外が起こった場合に備えて何かしら対応を考えておく、ということが重要です。  
その例外発生時に対応する処理のことを例外処理（エラーハンドリング）といいます。

では俗に言うエラー(Error)とは何が違うのでしょうか。

## Error 
アプリケーションでハンドリング可能な範囲を超えた、重大な問題です。  
エラーが起こった際には、Javaが速やかにプログラムを終了させるなどシステムの動作を継続できません。  
代表的なエラーは次のとおりです。  

|名称|説明|
|:-|:-|
|OutOfMemoryError|必要なメモリ領域を確保できない場合に発生|
|StackOverFlowError|スタック領域が限界を超えた場合に発生|
  
## 例外の種類

### 1.実行時例外(RuntimeException)  

実行時に起こった例外で必ずしも対処しなくても良い事象で、例外処理を書くか書かないかは状況によります。   
プログラムのバグの可能性も多いので、プログラムを修正することが正しい対応となる場合もあります。  
 
例えば、以下のような場合が該当します。   

|RuntimeExceptionの種類|事象の例|
|:-|:-|
|NullPointerException|インスタンスがある前提でプログラムは書かれているのに存在しなかった|
|NumberFormatException|Stringの数字をintの整数に変換しようとしたがそもそも数字ではなかった|  
|ArrayIndexOutOfBoundsException|不正なインデックスを使って配列にアクセスした|  

実行時例外はプログラム中で捕捉する必要のない例外であり、  
プログラムで捕捉しない場合にもコンパイルエラーは発生しません。



### 2.検査例外(Exception)

プログラム中で捕捉しなければならない例外で、  
プログラムで捕捉(catch)して処理するか、上位の呼び出し元に対して例外を発生させる(throwする)ことが必須となります。  
例外処理を書かなければ、コンパイルエラーとなるため例外処理の記述を強制されます。

Exceptionの代表的な例としては、ファイルを開こうとしたが対象のファイルが存在しなかった(FileNotFoundException)、などです。  
対象のファイルを開いた時にファイルが存在すれば、正常に処理が進められますが、  
なければ処理が進められない、つまり例外が発生したということになります。  
この場合は、無いことも予め予期できるので回復可能な例外となります。  　

例えば、以下のような場合が該当します。   

|Exceptionの種類|事象の例|
|:-|:-|
|FileNotFoundException| I/Oの処理に何らかの問題があった|
|IOException |ファイルが見つからなかった|
|ConnectException|ネットワークに接続できない|

## 例外処理の実装方法  
### 1. 例外をキャッチして処理を行う場合
<img src="./image/exception.png">  

### 1.1 try...catch...finally

```
try {
    例外がスローされる可能性がある処理;
} catch(例外1の型 変数) {
    例外1をキャッチしたときの処理;
    ※catch節は例外の数だけ指定できます。
} catch(例外2の型 変数) {
    例外2をキャッチしたときの処理;
                  ･
                  ･
} finally {
    必ず実行される処理;
}
```
※try節で処理中に例外が発生すると、以降の処理は行われず直ちにcatch節へ処理が移行します。


In [ ]:
// 例外処理
String className = "SampleTest";

try {
    System.out.println(Class.forName(className) + "は存在しています。"); 
    System.out.println("例外は発生しませんでした。"); 
} catch(ClassNotFoundException e) {
    System.out.println("例外が発生しました。");
    e.printStackTrace();
} finally {
    System.out.println("処理終了です");
}


### 1.2 try-with-resource 
tryブロックの中で何らかのリソース（InputStream、OutputStream、BufferedReader等）を扱う場合、  
Java7以前はfinallyブロックでcloseメソッドを呼び出すことで、  
tryブロック内の処理が正常終了したか異常終了したかに関わらずリソースが確実に閉じられることを保証していました。    
Java7以降ではtryブロックの開始時にAutoClosableインターフェースの実装クラスを宣言しておくと、    
そのtry...catchブロックの終了時の処理を行うcloseメソッドを自動的に呼び出してくれます。  
もしclose処理内で例外が発生してもtry文の中の例外がthrowされます。

- Java7以前
```
    InputStream in = new FileInputStream(src);
    try {
        OutputStream out = new FileOutputStream(dest);
        try {
            byte[] buf = new byte[100];
            int n;
            while ((n = in.read(buf)) >= 0) {
                out.write(buf, 0, n);
            }
        } finally {
            out.close();
        }
    } finally {
        in.close();
    }
 ```
 
- Java7以降
```
    try (InputStream inputStream = new FileInputStream(src); 
        OutputStream outputStream = new FileOutputStream(dest)) { // クローズすべきリソースが複数ある場合はセミコロンで区切る。
        byte[] buf = new byte[100];
        int n;
        while ((n = in.read(buf)) >= 0) {
            out.write(buf, 0, n);
        }
    }
```

### 2. thowsによる例外の伝搬
「throws」を使用すると、例外処理を行わず発生した例外をそのまま呼び出し元へ丸投げすることができます。  
以下の例では、処理中にFileNotFoundExceptionやIOExceptionが発生する可能性がありますが、  
本メソッド内では処理を行わずに、例外発生時にはそのまま呼び出し元へ例外を投げます。  
※呼び出し元のどこかで例外処理をする必要があります。 

```
public void read() throws FileNotFoundException, IOException {
  File file = new File("test.txt");
  FileReader reader = new FileReader(file);
  //何かしらファイルの処理が続く
  // ...
  reader.close();
  System.out.println("ファイル処理完了");
}
```

### 3. 明示的に例外をスローさせる場合
「throwステートメント 」を使用して、例外を明示的にスローさせることが出来ます。
```
throw　new 例外クラス名("エラーメッセージ");
```

# 問題
下記の例外処理を発生させるプログラムを書いてみましょう。  
- 2つ以上のメソッドを用意しましょう。
- 複数の種類の例外を補足する処理を用意しましょう。
- 例外の種類によって呼び出し元にthrowしてみましょう。
- 呼び出し元では細くした例外の種類別にメッセージを出力してみましょう。